In [1]:
from google.colab import drive
drive.mount('/content/MyDrive/')

Mounted at /content/MyDrive/


In [2]:
# Select TensorFlow 2.0
%tensorflow_version 2.x

In [3]:
# Install Weights and Biases
!pip install wandb

     |████████████████████████████████| 1.7 MB 5.8 MB/s 
     |████████████████████████████████| 144 kB 33.6 MB/s 
     |████████████████████████████████| 181 kB 44.9 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=cf9992406a00e79e24ddbf049345be9a0d861e4128a7b798e9a8a1f72625501a
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [4]:
# Imports
from tensorflow.keras.callbacks import TensorBoard

import wandb
from wandb.keras import WandbCallback

In [5]:
# Login to your W&B account
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
# Initialize your W&B project allowing it to sync with TensorBoard
wandb.init(project="tensorboardfall", sync_tensorboard=True)
config = wandb.config

wandb: Currently logged in as: taariqdb (use `wandb login --relogin` to force relogin)


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
df = pd.read_csv("/content/MyDrive/MyDrive/Allfalltypesnew.csv")

In [9]:
cols_to_scale = ['AccX','AccY','AccZ','GyrX','GyrY','GyrZ','EulerX','EulerY','EulerZ']

from sklearn.preprocessing import MinMaxScaler 
scaler = MinMaxScaler()
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

In [10]:
for column in df:
  print(f'{column} : {df[column].unique()}')

AccX : [0.49480926 0.49493433 0.49405879 ... 0.22726704 0.18098812 0.20425266]
AccY : [0.40885221 0.41010253 0.41135284 ... 0.967992   0.92498125 0.75493873]
AccZ : [0.47992495 0.48205128 0.48280175 ... 0.09093183 0.87166979 0.80863039]
GyrX : [0.36462854 0.3654879  0.36632579 ... 0.51076355 0.54015383 0.53501912]
GyrY : [0.50659189 0.50642448 0.50830787 ... 0.72410329 0.34290378 0.37295442]
GyrZ : [0.54384615 0.54302564 0.54276923 ... 0.28364103 0.70323077 0.82210256]
EulerX : [0.5529026  0.55368257 0.55454213 ... 0.46618277 0.47124461 0.82041609]
EulerY : [0.43238989 0.43245414 0.4326469  ... 0.76136473 0.76852893 0.87277926]
EulerZ : [0.64456048 0.64451272 0.64448088 ... 0.29797192 0.30650451 0.31171002]
TrialID : [1 2 3 4 5 6 7]
TaskID : [20 21 22 23 24 25 26 27 28 29 30 31 32 33 34]
Subject : ['SA06' 'SA07' 'SA08' 'SA09' 'SA10' 'SA11' 'SA12' 'SA13' 'SA14' 'SA15'
 'SA17' 'SA24' 'SA29' 'SA35' 'SA36' 'SA37' 'SA38' 'SA30' 'SA16' 'SA21'
 'SA25' 'SA31' 'SA18' 'SA27' 'SA19' 'SA22' 'SA26'

In [11]:
df = pd.get_dummies(data = df, columns=['DescriptionID'])
df = pd.get_dummies(data = df, columns=['FallTypes'])
df.columns

Index(['AccX', 'AccY', 'AccZ', 'GyrX', 'GyrY', 'GyrZ', 'EulerX', 'EulerY',
       'EulerZ', 'TrialID', 'TaskID', 'Subject', 'Description', 'FTLabels',
       'DescriptionID_1', 'DescriptionID_2', 'DescriptionID_3',
       'DescriptionID_4', 'DescriptionID_5', 'DescriptionID_6',
       'DescriptionID_7', 'DescriptionID_8', 'DescriptionID_9',
       'DescriptionID_10', 'DescriptionID_11', 'DescriptionID_12',
       'DescriptionID_13', 'DescriptionID_14', 'DescriptionID_15',
       'FallTypes_Backward', 'FallTypes_Dampen', 'FallTypes_Forward',
       'FallTypes_Lateral'],
      dtype='object')

In [12]:
X = df[['AccX', 'AccY', 'AccZ', 'GyrX', 'GyrY', 'GyrZ', 'EulerX', 'EulerY','EulerZ',
       'DescriptionID_1', 'DescriptionID_2', 'DescriptionID_3',
       'DescriptionID_4', 'DescriptionID_5', 'DescriptionID_6',
       'DescriptionID_7', 'DescriptionID_8', 'DescriptionID_9',
       'DescriptionID_10', 'DescriptionID_11', 'DescriptionID_12',
       'DescriptionID_13', 'DescriptionID_14', 'DescriptionID_15',]]
X = np.asarray(X)

In [13]:
y = df[['FallTypes_Backward', 'FallTypes_Dampen', 'FallTypes_Forward','FallTypes_Lateral']]
y = np.asarray(y)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [15]:
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import keras
from keras.models import Sequential
from keras.layers import Dense
import warnings

In [16]:
X.shape , y.shape

((173649, 24), (173649, 4))

In [17]:
from sklearn.metrics import confusion_matrix
from datetime import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import itertools
import io

In [18]:
labels = ['Backward','Dampen','Forward','Lateral']
classlabel = ['Forward','Lateral','Backward','Dampen']

In [19]:
model = Sequential([
                    Dense(11, input_shape=(24,), activation='relu'),
                    Dense(4,activation='sigmoid')          
])
                    
# Compiling the ANN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(X_train,y_train,epochs=200,validation_split=0.25,batch_size=40,verbose=2,callbacks=[WandbCallback(),TensorBoard(log_dir=wandb.run.dir)])

Epoch 1/200


wandb: WARNING When using several event log directories, please call wandb.tensorboard.patch(root_logdir="...") before wandb.init


2605/2605 - 14s - loss: 0.0746 - accuracy: 0.9562 - val_loss: 0.0021 - val_accuracy: 1.0000 - _timestamp: 1648794776.0000 - _runtime: 24.0000 - 14s/epoch - 5ms/step
Epoch 2/200
2605/2605 - 9s - loss: 8.3057e-04 - accuracy: 1.0000 - val_loss: 2.6225e-04 - val_accuracy: 1.0000 - _timestamp: 1648794785.0000 - _runtime: 33.0000 - 9s/epoch - 4ms/step
Epoch 3/200
2605/2605 - 9s - loss: 1.2882e-04 - accuracy: 1.0000 - val_loss: 5.1899e-05 - val_accuracy: 1.0000 - _timestamp: 1648794795.0000 - _runtime: 43.0000 - 9s/epoch - 4ms/step
Epoch 4/200
2605/2605 - 10s - loss: 2.6966e-05 - accuracy: 1.0000 - val_loss: 1.1604e-05 - val_accuracy: 1.0000 - _timestamp: 1648794804.0000 - _runtime: 52.0000 - 10s/epoch - 4ms/step
Epoch 5/200
2605/2605 - 10s - loss: 6.1652e-06 - accuracy: 1.0000 - val_loss: 2.7339e-06 - val_accuracy: 1.0000 - _timestamp: 1648794815.0000 - _runtime: 63.0000 - 10s/epoch - 4ms/step
Epoch 6/200
2605/2605 - 10s - loss: 1.4715e-06 - accuracy: 1.0000 - val_loss: 6.6748e-07 - val_accu

In [20]:
#using lstm model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
pred=model.predict(X_test)
accuracy=model.evaluate(X_test,y_test)
print("accuracy is {}".format(accuracy[1]))
wandb.sklearn.plot_confusion_matrix(y_test.argmax(axis=1), pred.argmax(axis=1),labels)
wandb.termlog('Logged confusion matrix.')
wandb.sklearn.plot_class_proportions(y_train.argmax(axis=1), y_test.argmax(axis=1),classlabel)
wandb.termlog('Logged class proportions.')

1086/1086 [==============================] - 3s 3ms/step - loss: 1.4333e-09 - accuracy: 1.0000
accuracy is 1.0


wandb: Logged confusion matrix.
wandb: Logged class proportions.
